In [1]:
import jupyter_black
import dotenv

dotenv.load_dotenv()
jupyter_black.load()

In [2]:
from typing import Any
from baml_agents._baml_client_proxy._hooks._on_before_call_hook import (
    OnBeforeCallHookContext,
    OnBeforeCallHookSync,
)
from baml_client.sync_client import b
from baml_client import types
from baml_agents import with_hooks
from f01_chat import get_args_block_str


class BamlTestGeneratorHook(OnBeforeCallHookSync):
    def __init__(self):
        self.counter = 0
        self.baml_test_source_code = ""

    def on_before_call(self, params: dict[str, Any], ctx: OnBeforeCallHookContext):
        self.counter += 1
        test_name = f"llm_call_{self.counter}"
        args_block_str = get_args_block_str(params)
        self.baml_test_source_code += f"""\
test {test_name} {{
  functions [{ctx.baml_function_name}]
  args {{
{args_block_str}
  }}
}}\n\n"""

In [3]:
test_hook = BamlTestGeneratorHook()
b = with_hooks(b, [test_hook])

b.ChatWithLLM(
    messages=[
        types.LLMChatMessage(role="user", content="What's the weather like?"),
    ]
)
b.ChatWithLLM(
    messages=[
        types.LLMChatMessage(role="user", content="What's the weather like?"),
        types.LLMChatMessage(role="assistant", content="It's sunny."),
        types.LLMChatMessage(role="user", content="Do you like sunny weather?"),
    ]
)

print(test_hook.baml_test_source_code)

test llm_call_1 {
  functions [ChatWithLLM]
  args {
    messages [
      {
        role "user"
        content "What's the weather like?"
      }
    ]
  }
}

test llm_call_2 {
  functions [ChatWithLLM]
  args {
    messages [
      {
        role "user"
        content "What's the weather like?"
      },
      {
        role "assistant"
        content "It's sunny."
      },
      {
        role "user"
        content "Do you like sunny weather?"
      }
    ]
  }
}


